# Package

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import requests
import re

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# function(comment,post)

In [12]:
def get_user_post(user):
    # function
    def get_info_from_url(URL,post_number):
        def get_post_article(soup):
            try:
                soup.find('div',{'itemprop':'articleBody','class':'e7-main-content'}).text.strip('\n')
            except AttributeError:
                return None
            else:
                return soup.find('div',{'itemprop':'articleBody','class':'e7-main-content'}).text.strip('\n')
        board = []
        # author = []
        date = []
        push = []
        count = []
        article = []
        i = 1
        for url in URL:
            print('current progress: {} posts,total {} posts.'.format(i,post_number))
            i = i + 1
            request_html = requests.get(url)
            soup = BeautifulSoup(request_html.text,'html.parser')
            board.append(soup.find('span',{'class':'e7-board-name-standalone'}).getText())
            # author.append(soup.find('span',{'itemprop':'name'}).getText()+soup.find('span',{'class':'e7-head-small'}).getText())
            date.append(soup.find('time',{'itemprop':'datePublished'}).getText())
            push.append(int(soup.find_all('span',{'itemprop':'userInteractionCount'})[0].getText())-int(soup.find_all('span',{'itemprop':'userInteractionCount'})[1].getText()))
            count.append(soup.find_all('span',{'class':'e7-head-content'})[4].getText().partition('則')[0])
            article.append(get_post_article(soup))
        INFO = pd.DataFrame({'版面':board,'日期':date,'內文':article,'推文數':push,'留言數':count,'網址':URL})
        return INFO
    # get url
    URL = []
    url = 'https://www.pttweb.cc/user/{}?t=article&page=0'.format(user)
    while True:
        request_html = requests.get(url)
        soup = BeautifulSoup(request_html.text,'html.parser')
        titles = soup.find_all("div", {"class": "e7-left"})
        for title in titles:
            allnode_of_a = title.find('a')
            URL.append('https://www.pttweb.cc'+ allnode_of_a.get('href'))
        # last page break
        post_info = soup.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
        post_number = int([s for s in re.findall(r'-?\d+\.?\d*',post_info)][-1])
        if post_number <= int(len(URL)):
            break
        #next page
        allnode_of_div = soup.find('div',{'class':'e7-pagination'})
        try:
            url = 'https://www.pttweb.cc'+allnode_of_div.find_all('a')[-2].get('href')
        except IndexError:
            break
    ALL_INFO = get_info_from_url(URL,post_number)
    return ALL_INFO

def get_user_comment(user):
    # function
    def get_comment_date(text):
        if len(text)>12:
            return text[-11:]
        else:
            return text
    def get_post_author(ii,user):
        try:
            ii.find_all('span',{'class':'thread-list-author'})[1].getText()
        except IndexError:
            return user
        else:
            return ii.find_all('span',{'class':'thread-list-author'})[1].getText()
    # initial setting
    url = 'https://www.pttweb.cc/user/{}?t=message&page=0'.format(user)
    comment = []
    comment_date = []
    post_title = []
    post_board = []
    post_comment_count = []
    post_push = []
    post_auther = []
    post_date = []
    while True:
        request_html = requests.get(url)
        soup = BeautifulSoup(request_html.text,'html.parser')
        # information
        allnode = soup.find_all('div',{'class':'thread-item'})
        post_info = soup.find('div',{'class':'headline e7-block-title pl-3 mt-4'}).text
        post_number = int([s for s in re.findall(r'-?\d+\.?\d*',post_info)][-1])
        for node in allnode:
            string = node.find('span',{'class':'thread-popularity px-2'}).getText()
            number = [int(s) for s in re.findall(r'-?\d+\.?\d*', string)]
            for i in node.find_all('div',{'class':'e7-top mt-2'}):
                comment.append(i.find('span',{'class':'yellow--text text--darken-2'}).getText()[1:])
                comment_date.append(get_comment_date(i.find('span',{'class':'ml-3 grey--text text--lighten-1'}).getText()))
                post_title.append(node.find('span',{'class':'thread-title'}).getText())
                post_board.append(node.find('span',{'class':'thread-list-board'}).getText()[1:-1].strip())
                post_comment_count.append(number[0])
                post_push.append(number[1])
                post_auther.append(get_post_author(node,user))
                post_date.append(node.find('span',{'class':'thread-posttime'}).getText())
                print('current progress: {} comments,total {} comments.'.format(int(len(post_board)),post_number))
        # last page break
        if int(len(post_board)) >= post_number:
            break
        #next page
        allnode_of_a = soup.find('div',{'class':'e7-pagination'})
        try:
            url = 'https://www.pttweb.cc' + allnode_of_a.find_all('a')[-2].get('href')
        except IndexError:
            break
    INFO = pd.DataFrame({'留言':comment,'留言日期':comment_date,'文章標題':post_title,'文章版面':post_board,'文章留言數':post_comment_count,
    '文章推數':post_push,'文章作者':post_auther,'發文日期':post_date})
    return INFO


# Get Comment

In [60]:
user = 'xxxxgay'
ALL_COMMENT = get_user_comment(user)
print(ALL_COMMENT)

                        留言         留言日期                            文章標題  \
0        .................  06/03 19:56         Re: [問車] 80-100萬CUV選擇障礙   
1            我啥時變民進黨了？（舉手）  09/20 07:14            [討論] 所以民進黨要開始打高虹安論文嗎   
2          我有收錢呀，7000元車馬費。  09/20 07:36            [討論] 所以民進黨要開始打高虹安論文嗎   
3              有種你就屌打我看看阿！  12/17 20:09      [新聞] 痛批基隆輕軌是騙局 柯文哲：基捷400多億   
4        你要不要先解釋英翻中為啥一字不差？  09/07 09:08  Re: [討論] <民營化策略> 論文改編成書，絕無抄襲~~   
...                    ...          ...                             ...   
17044            最近好忙好忙 XD  5.194 12/28         [秘技]隱身斗篷-穿越時空攻塔術(BUG教學)   
17045       我忙著找男人陪我跨年啦 XD  5.194 12/29         [秘技]隱身斗篷-穿越時空攻塔術(BUG教學)   
17046         這次說不定改鎖創杖 XD  5.194 12/29         [秘技]隱身斗篷-穿越時空攻塔術(BUG教學)   
17047   當版主可以騙到很多男人嗎?(小聲問)  5.194 12/31                       [公告] BM徵召   
17048  唔...那我當版主,在這裡開後宮 XD  5.194 12/31                       [公告] BM徵召   

               文章版面  文章留言數  文章推數      文章作者              發文日期  
0               car     77    49  jo

# Get Post Information

In [105]:
user = 'xxxxgay'
ALL_INFO = get_user_post(user)
print(ALL_INFO)

             版面                日期  推文數  留言數  \
0          book  2022/09/07 09:27    8   14   
1          book  2022/09/07 01:23    1    1   
2          book  2022/09/06 15:14   14   21   
3     Gossiping  2022/09/02 00:00 -141  226   
4     Gossiping  2022/08/29 00:07 -310  458   
...         ...               ...  ...  ...   
1815    Lineage  2005/05/04 13:50    0   33   
1816    Lineage  2005/03/31 12:11    4    5   
1817    Lineage  2005/03/20 07:18    0    0   
1818    Lineage  2005/01/28 13:13    1    1   
1819    Lineage  2004/12/28 21:06   15   15   

                                                     網址  
0     https://www.pttweb.cc/bbs/book/M.1662514030.A.751  
1     https://www.pttweb.cc/bbs/book/M.1662484998.A.C14  
2     https://www.pttweb.cc/bbs/book/M.1662448446.A.90E  
3     https://www.pttweb.cc/bbs/Gossiping/M.16620480...  
4     https://www.pttweb.cc/bbs/Gossiping/M.16617028...  
...                                                 ...  
1815  https://www.pttweb.cc/b

# Output

In [4]:
ALL_COMMENT.to_csv('C:/Users/USER/OneDrive/Documents/RA/{}留言.csv'.format(user),index=False,encoding="utf_8_sig")

In [108]:
ALL_INFO.to_csv('{}貼文.csv'.format(user),index=False,encoding="utf_8_sig")

In [9]:
USER_SET = [#'liang691206',
'yoyoflag','stevencool','bezbol','coober']
for i in USER_SET:
    ALL_INFO = get_user_post(i)
    ALL_INFO.to_csv('{}貼文.csv'.format(i),index=False,encoding="utf_8_sig")
    print('{}done'.format(i))

yoyoflagdone
stevencooldone
bezboldone
cooberdone


# Other

In [14]:
user = 'liang691206'
get_user_post(user)

current progress: 1 posts,total 5399 posts.
current progress: 2 posts,total 5399 posts.
current progress: 3 posts,total 5399 posts.
current progress: 4 posts,total 5399 posts.
current progress: 5 posts,total 5399 posts.
current progress: 6 posts,total 5399 posts.
current progress: 7 posts,total 5399 posts.
current progress: 8 posts,total 5399 posts.
current progress: 9 posts,total 5399 posts.
current progress: 10 posts,total 5399 posts.
current progress: 11 posts,total 5399 posts.
current progress: 12 posts,total 5399 posts.
current progress: 13 posts,total 5399 posts.
current progress: 14 posts,total 5399 posts.
current progress: 15 posts,total 5399 posts.
current progress: 16 posts,total 5399 posts.
current progress: 17 posts,total 5399 posts.
current progress: 18 posts,total 5399 posts.
current progress: 19 posts,total 5399 posts.
current progress: 20 posts,total 5399 posts.
current progress: 21 posts,total 5399 posts.
current progress: 22 posts,total 5399 posts.
current progress: 2

,版面,日期,內文,推文數,留言數,網址
0,WomenTalk,2022/10/23 22:35,※ 引述《dream20225 (sue )》之銘言：\n\n看到這個\n\n就有借過朋友錢...,2,7,https://www.pttweb.cc/bbs/WomenTalk/M.16665357...
1,WomenTalk,2022/10/23 19:52,※ 引述《bruse11 (刀鋒33)》之銘言：\n\n沒什麼漲啊\n\n大概十幾年前\n\...,2,3,https://www.pttweb.cc/bbs/WomenTalk/M.16665259...
2,Boy-Girl,2022/10/21 21:09,常常看到台灣女森\n\n要求另外一半年薪百萬\n\n說什麼人均GDP都三萬多鎂了\n\n年薪...,4,22,https://www.pttweb.cc/bbs/Boy-Girl/M.166635779...
3,Gossiping,2022/10/21 13:21,民法有規定\n\n夫妻有\n\n愛愛義務\n\n守貞義務\n\n互相代理義務\n\n當然有相...,0,0,https://www.pttweb.cc/bbs/Gossiping/M.16663296...
4,HatePolitics,2022/10/21 13:14,※ 引述《jimlexus (中華民國空軍４％嘴砲營)》之銘言：\n\n怎麼有人會覺得新竹是...,1,10,https://www.pttweb.cc/bbs/HatePolitics/M.16663...
...,...,...,...,...,...,...
5395,historia,2013/09/04 06:04,支那是貶義?\n\n應該是共產黨這幾年炒出來的話題吧\n\n其它孫文 黃興所自稱支那不再多...,5,17,https://www.pttweb.cc/bbs/historia/M.137824587...
5396,historia,2013/08/29 18:55,※ 引述《in09 ()》之銘言：\n: ※ 引述《Nakata0911 (夏亞 阿茲納布 ...,12,24,https://www.pttweb.cc/bbs/historia/M.137777374...
5397,movie,2013/07/27 01:53,這部電影根本就是番外篇\n\n而且瞎得離譜\n\n劇情有點沉悶\n\nbug很多\n\n1....,8,17,https://www.pttweb.cc/bbs/movie/M.1374861197.A...
5398,movie,2013/07/14 17:45,環太平洋\n\n機器人外型 鐵人28號\...,7,15,https://www.pttweb.cc/bbs/movie/M.1373795153.A...
